## Import libraries

In [ ]:
!pip install selenium webdriver_manager

In [ ]:
from bs4 import BeautifulSoup
import re, os, logging, time, argparse, unicodedata, html5lib, requests
import pandas as pd
import numpy as np
from openpyxl import Workbook
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

## Helper functions
1. **parse_and_trim()** -> Parses and trims HTML content by removing all attributes from HTML tags and removing line break tags from the content
2. **remove_multiple_spaces()** -> Replace multiple spaces with a single space in a string
3. **find_qtr_date()** -> Extract and format a quarterly date from text content

In [ ]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'lxml')
    else:
        soup = BeautifulSoup(content, 'lxml')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup
    
# def find_qrt_date(content):
#     qtr_date = content.find_all(text=re.compile(
#         r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))
#     qtr_match = re.search(
#         r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
#     if qtr_match is None:
#         qtr_match = qtr_match = re.search(
#             r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
#     return remove_multiple_spaces(str(qtr_match.group()))

In [ ]:
def remove_multiple_spaces(string):
    pattern = r'\s+'
    replaced_string = re.sub(pattern, ' ', string)
    return replaced_string

In [ ]:
def find_qrt_date(content):
#     qtr_date = content.find_all(string=re.compile(
#         r'for\s+(the\s+)?(fiscal\s+)?(year|quarter(ly)?)\s+ended\s+', re.IGNORECASE))
    
    qtr_date = content.find_all(text=re.compile(
        r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))

    # Check if any matching elements were found
    if not qtr_date:
        return None
    
    qtr_match = re.search(
        r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
    if qtr_match is None and len(qtr_date) > 1:
        qtr_match = re.search(
            r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
    
    if qtr_match:
        return remove_multiple_spaces(str(qtr_match.group()))
    else:
        return None

## Retrieve filing links

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.sec.gov/edgar/browse/?CIK=1655887'
driver.get(url)
html_content = driver.page_source
if not os.path.exists('htmls'):
    os.mkdir('htmls')
with open(os.path.join('htmls', url.split("=")[-1]+".html"), "w", encoding='utf-8') as file:
    file.write(html_content)
dfs = pd.read_html(html_content)

if not os.path.exists('csv'):
    os.mkdir('csv')
for i, df in enumerate(dfs):
    df.to_csv(os.path.join('csv', url.split("=")[-1]+f"_link_table_{i}.csv"))

h5_tags = driver.find_elements_by_tag_name("h5")

for h5_tag in h5_tags:
    if h5_tag.text == "[+] 10-K (annual reports) and 10-Q (quarterly reports)":
        # Click on the h5 tag.
        h5_tag.click()
        break


xpath = '//button[text()="View all 10-Ks and 10-Qs"]'
element = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, xpath)))
driver.execute_script("arguments[0].click();", element)

conditions = '@data-original-title="Open document" and contains(@href, "Archive") and not(contains(@href, "index")) and not(contains(@href, "xml"))'
table = driver.find_elements_by_css_selector('div.dataTables_scroll')
links = table[0].find_elements_by_xpath(f'//td//a[{conditions}]')


logging.debug(
    f"LINKS - {len([link.get_attribute('innerHTML') for link in links])}")
df = pd.read_html(table[0].get_attribute('innerHTML'))[-1]
filing_date = df['Reporting date']


logging.debug(f"DATES - {len(filing_date)}")

with open(os.path.join('urls', url.split("=")[-1]+".txt"), 'w') as url_out:
    for a, date in zip(links, filing_date):
        url_out.write('\n%s %s' %
                      (date.split("View")[0], a.get_attribute('href')))
        logging.debug('\n%s %s' %
                      (date.split("View")[0], a.get_attribute('href')))
driver.close()

In [ ]:
table = pd.read_csv('../../../../../Downloads/EDGAR Entity Landing Page.csv')
table['Reporting date'] = pd.to_datetime(table['Reporting date'], format='%Y-%m-%d')

In [ ]:
table.info()

In [ ]:
url_table = pd.read_csv('../Extract_link/urls/1655887.txt',
                        names=['Reporting date', 'url'], delim_whitespace=True)
url_table['Reporting date'] = pd.to_datetime(url_table['Reporting date'], format='%Y-%m-%d')

In [ ]:
table = table.merge(url_table, on='Reporting date')
table.to_csv('1655887.csv')
table = table.drop(table[table['Form description'].str.contains(
    'amendment', case=False)].index).reset_index(drop=True)
table['Reporting date'] = table['Reporting date'].astype(str)

## Preprocess the filings

In [ ]:
# Assign headers
headers = {
    'User-Agent': 'Blue Owl Capital Corp II'
}

# Extract to a dataframe
df = pd.read_excel("/kaggle/input/1655887/filings_links.xlsx")
df.head()

In [ ]:
# Drops all the amendment filing
# Keep only Qs & Ks
df = df.drop(df[df['Form description'].str.contains(
    'amendment', case=False)].index).reset_index(drop=True)
df['Reporting date'] = pd.to_datetime(df['Reporting date'])
df.head()

In [ ]:
# Convert to datetime format
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
    
for col in date_columns:
    df[col] = df[col].dt.strftime("%B %d, %Y")
    
df.head()

In [ ]:
# Last check
df.info()

## Tables Extraction

In [ ]:
qtr_dates = []
for index, url in enumerate(df['Filings URL']):
    response = requests.get(url, headers=headers)
    print(response)
    content = parse_and_trim(response.content, 'HTML')
    print(content)
    qtr_date = find_qrt_date(content)
    print(qtr_date)
    if qtr_date is not None:
        qtr_date = qtr_date.replace(',', '').strip()
    print(qtr_date)
    qtr_dates.append(qtr_date)

# Add quarter dates column to each filing
df['qtr_date'] = qtr_dates